In [1]:
#from bs4 import BeautifulSoup
#import requests
import pandas as pd
import unidecode
from pathlib import Path
import numpy as np

Formato do float

In [17]:
pd.options.display.float_format = '{:,.2f}'.format

# Doações

In [2]:
cand_doacoes = pd.read_csv("doacoes/receitas_candidatos_2014_brasil.txt",sep=';', header=0, encoding = 'latin_1', converters={'CPF do candidato': lambda x: str(x), 'CPF/CNPJ do doador': lambda x: str(x)}, decimal=',')

In [3]:
# Apaga colunas que não serão usadas
del cand_doacoes['Cód. Eleição']
del cand_doacoes['Desc. Eleição']
del cand_doacoes['CNPJ Prestador Conta']
del cand_doacoes['Data e hora']
del cand_doacoes['Sequencial Candidato']
del cand_doacoes['Numero candidato']
del cand_doacoes['Numero Recibo Eleitoral']
del cand_doacoes['Numero do documento']
del cand_doacoes['Nome do doador (Receita Federal)']
del cand_doacoes['Sigla UE doador']
del cand_doacoes['Número partido doador']
del cand_doacoes['Número candidato doador']
del cand_doacoes['Cod setor econômico do doador']
del cand_doacoes['Setor econômico do doador']
del cand_doacoes['Data da receita']
del cand_doacoes['Setor econômico do doador originário']
del cand_doacoes['Nome do doador originário (Receita Federal)']

In [4]:
cand_doacoes_d_2014 = cand_doacoes[(cand_doacoes['Cargo'] == 'Deputado Estadual') & (cand_doacoes['UF'] == 'SP')]

In [5]:
#cand_doacoes_d_2014 = cand_doacoes_d_2014[(cand_doacoes_d_2014['CPF/CNPJ do doador'] != 0.0)]

In [6]:
#cand_doacoes_d_2014 = cand_doacoes_d_2014[pd.notnull(cand_doacoes_d_2014['CPF/CNPJ do doador'])]

In [5]:
cand_doacoes_d_2014.columns = ['uf', 'partido_eleicao', 'cargo', 'nome_completo', 'cpf', 'cpf_cnpj_doador', 'nome_doador', 'valor', 'tipo_receita', 'fonte_recurso', 'especie_recurso', 'descricao', 'cpf_cnpj_doador_originario', 'nome_doador_originario', 'tipo_doador_originario']

In [6]:
def limpeza1(linha):
    documento = linha["cpf_cnpj_doador"]
    tipo = linha["tipo_receita"]
    if len(documento) > 11 and tipo != "Recursos de outros candidatos/comitês":
        return documento[:8]
    elif tipo == "Recursos de outros candidatos/comitês":
        return "Recursos de outros candidatos/comitês"
    else:
        return "documento_cpf"

def limpeza2(linha):
    documento = linha["cpf_cnpj_doador_originario"]
    if len(documento) > 11 and documento != "#NULO" and documento != None:
        return documento[:8]
    elif documento == "#NULO" or documento == None:
        return "#NULO"
    else:
        return "documento_cpf"

In [7]:
cand_doacoes_d_2014['cnpj_doador_raiz'] = cand_doacoes_d_2014.apply(limpeza1, axis = 1)
cand_doacoes_d_2014['cnpj_doador_originario_raiz'] = cand_doacoes_d_2014.apply(limpeza2, axis = 1)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
cand_doacoes_d_2014.rename(columns={'valor':'valor_doado'}, inplace=True)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [18]:
cand_doacoes_d_2014.reset_index().head(5)

,index,uf,partido_eleicao,cargo,nome_completo,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,especie_recurso,descricao,cpf_cnpj_doador_originario,nome_doador_originario,tipo_doador_originario,cnpj_doador_raiz,cnpj_doador_originario_raiz
0,23,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,58809203000103,BRASILIAN COLOR I T,"2,500.00",Recursos de pessoas jurídicas,Nao especificado,Transferência eletrônica,#NULO,#NULO,#NULO,#NULO,58809203,#NULO
1,51,SP,PSDB,Deputado Estadual,JOAO CARLOS CARAMEZ,41352009820,43483247000119,HIDROCONSULT CONSULTORIA ESTUDOS E PROJETOS LTDA,"3,000.00",Recursos de pessoas jurídicas,Nao especificado,Cheque,#NULO,#NULO,#NULO,#NULO,43483247,#NULO
2,114,SP,PT do B,Deputado Estadual,PAULO ROBERTO ROSENO JUNIOR,05488774831,20557696000169,JORDÃO MOISES FABREGA,447.50,Recursos de outros candidatos/comitês,Outros Recursos nao descritos,Estimado,REFERENTE A DOBRADA DE 15 CAVALETES E 1 PLACA ...,00639041000122,INMETRA MEDICINA E SEGURANÇA DO TRABALHO LTDA,J,Recursos de outros candidatos/comitês,00639041
3,220,SP,PV,Deputado Estadual,BRUNO AREVALO GANEM,35205827831,07334221000145,WBB CONSTRUÇÕES LTDA,840.00,Recursos de pessoas jurídicas,Nao especificado,Estimado,CESSÃO DO VEÍCULO VW KOMBI 2010 - PLACAS DSS-5115,#NULO,#NULO,#NULO,07334221,#NULO
4,221,SP,PSC,Deputado Estadual,MARIA HELENA JURADO MELILLO,22678150879,20559333000162,ELEIÇÃO 2014 ROBERTO MIGUEL REY JUNIOR DEPUTAD...,220.00,Recursos de outros candidatos/comitês,Outros Recursos nao descritos,Estimado,40 POR CENTO DE 50 CONVITES 14X21 CM PAPEL 300...,#NULO,#NULO,#NULO,Recursos de outros candidatos/comitês,#NULO


# Candidatos

In [2]:
def _reader_2014(fname):
    return pd.read_csv(fname, sep=';', header=None, encoding='latin_1', converters={13: lambda x: str(x), 11: lambda x: str(x)})

In [3]:
folder = Path("candidatos_2014")
candidatos_2014 = pd.concat([
    _reader_2014(txt)
    for txt in folder.glob("*.txt")
])

In [4]:
# Apaga as colunas que não serão usadas
del candidatos_2014[0]
del candidatos_2014[1]
del candidatos_2014[2]
del candidatos_2014[3]
del candidatos_2014[4]
del candidatos_2014[6]
del candidatos_2014[7]
del candidatos_2014[8]
del candidatos_2014[12]
del candidatos_2014[15]
del candidatos_2014[16]
del candidatos_2014[17]
del candidatos_2014[19]
del candidatos_2014[20]
del candidatos_2014[21]
del candidatos_2014[22]
del candidatos_2014[23]
del candidatos_2014[24]
del candidatos_2014[25]
del candidatos_2014[26]
del candidatos_2014[27]
del candidatos_2014[28]
del candidatos_2014[29]
del candidatos_2014[30]
del candidatos_2014[31]
del candidatos_2014[32]
del candidatos_2014[33]
del candidatos_2014[34]
del candidatos_2014[35]
del candidatos_2014[36]
del candidatos_2014[37]
del candidatos_2014[38]
del candidatos_2014[39]
del candidatos_2014[40]
del candidatos_2014[41]
del candidatos_2014[42]
del candidatos_2014[43]
del candidatos_2014[45]

In [5]:
candidatos_2014.columns = ['uf', 'cargo', 'nome_completo', 'sequencial', 'cpf', 'nome_urna', 'partido_eleicao', 'situacao']

In [6]:
deputadossp_eleicao2014 = candidatos_2014[(candidatos_2014['cargo'] == 'DEPUTADO ESTADUAL') & (candidatos_2014['uf'] == 'SP')]

In [7]:
deputadossp_eleicao2014.to_csv('resultados/deputados_queconcorreram_2014_sp.csv', index=False) 

Carrega nomes de deputados da atual legislatura

In [10]:
eleitos_d_2014 = pd.read_csv('resultados/deputados_selecionados_final.csv', converters={'sequencial': lambda x: str(x), 'cpf': lambda x: str(x), 'codigo_unico': lambda x: str(x) })

In [11]:
eleitos_d_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
codigo_unico       120 non-null object
nome_deputado      120 non-null object
uf                 120 non-null object
nome_completo      120 non-null object
sequencial         120 non-null object
cpf                120 non-null object
nome_urna          120 non-null object
partido_eleicao    120 non-null object
situacao           120 non-null object
dtypes: object(9)
memory usage: 8.5+ KB


In [12]:
eleitos_d_doadores = pd.merge(cand_doacoes_d_2014, eleitos_d_2014, left_on='cpf', right_on='cpf')

In [13]:
eleitos_d_doadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15675 entries, 0 to 15674
Data columns (total 25 columns):
uf_x                           15675 non-null object
partido_eleicao_x              15675 non-null object
cargo                          15675 non-null object
nome_completo_x                15675 non-null object
cpf                            15675 non-null object
cpf_cnpj_doador                15675 non-null object
nome_doador                    15675 non-null object
valor_doado                    15675 non-null float64
tipo_receita                   15675 non-null object
fonte_recurso                  15675 non-null object
especie_recurso                15675 non-null object
descricao                      15675 non-null object
cpf_cnpj_doador_originario     15675 non-null object
nome_doador_originario         15675 non-null object
tipo_doador_originario         15675 non-null object
cnpj_doador_raiz               15675 non-null object
cnpj_doador_originario_raiz    15675 non-n

In [14]:
eleitos_d_doadores.to_csv('resultados/doacoes_deputados_alesp_2014.csv', index=False, decimal = ',')

# Cruza com despesas de deputados

In [15]:
eleitos_d_doadores = pd.read_csv("resultados/doacoes_deputados_alesp_2014.csv", sep=',',encoding = 'utf-8', 
                                 converters={'cpf_cnpj_doador': lambda x: str(x), 
                                             'cpf': lambda x: str(x),  
                                             'cpf_cnpj_doador_originario': lambda x: str(x), 
                                             'cnpj_doador_raiz': lambda x: str(x), 
                                             'cnpj_doador_originario_raiz': lambda x: str(x), 
                                             'sequencial': lambda x: str(x),
                                             'codigo_unico': lambda x: str(x),
                                            }, decimal=',')

In [16]:
eleitos_d_doadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15675 entries, 0 to 15674
Data columns (total 25 columns):
uf_x                           15675 non-null object
partido_eleicao_x              15675 non-null object
cargo                          15675 non-null object
nome_completo_x                15675 non-null object
cpf                            15675 non-null object
cpf_cnpj_doador                15675 non-null object
nome_doador                    15675 non-null object
valor_doado                    15675 non-null float64
tipo_receita                   15675 non-null object
fonte_recurso                  15675 non-null object
especie_recurso                15675 non-null object
descricao                      15675 non-null object
cpf_cnpj_doador_originario     15675 non-null object
nome_doador_originario         15675 non-null object
tipo_doador_originario         15675 non-null object
cnpj_doador_raiz               15675 non-null object
cnpj_doador_originario_raiz    15675 non-n

Total valor doado para os 120

In [20]:
"${:,.2f}".format(eleitos_d_doadores.valor_doado.sum())

'$124,484,320.63'

Busca de doador pelo CNPJ raiz

In [21]:
nome = eleitos_d_doadores[(eleitos_d_doadores['cnpj_doador_raiz'] == '43283811')]
nome

,uf_x,partido_eleicao_x,cargo,nome_completo_x,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,cnpj_doador_raiz,cnpj_doador_originario_raiz,codigo_unico,nome_deputado,uf_y,nome_completo_y,sequencial,nome_urna,partido_eleicao_y,situacao
2432,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,#NULO,10600,FERNANDO CAPEZ,SP,FERNANDO CAPEZ,250000001510,FERNANDO CAPEZ,PSDB,ELEITO POR QP
6927,SP,PC do B,Deputado Estadual,LECI BRANDÃO DA SILVA,18264590730,43283811000150,KALUNGA COMÉRCIO E INDÚSTRIA GRÁFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,#NULO,12420,LECI BRANDAO,SP,LECI BRANDÃO DA SILVA,250000000071,LECI BRANDAO,PC do B,ELEITO POR QP
11424,SP,PV,Deputado Estadual,EDSON DE OLIVEIRA GIRIBONI,98361325891,43283811000150,KALUNGA COMÉRCIO E INDÚSTRIA GRÁFICA LTDA,"5,270.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,#NULO,10597,EDSON GIRIBONI,SP,EDSON DE OLIVEIRA GIRIBONI,250000001019,EDSON GIRIBONI,PV,ELEITO POR QP


In [24]:
despesas = pd.read_csv("resultados/despesas_atuais_final.csv", sep=',',encoding = 'utf-8', 
                       converters={'CNPJ': lambda x: str(x), 
                                   'cnpj_raiz_fornecedor': lambda x: str(x), 
                                   'Ano': lambda x: str(x), 
                                   'codigo_unico': lambda x: str(x), 
                                   'cpf': lambda x: str(x), 
                                   'sequencial': lambda x: str(x)
                                  }, decimal=',')

In [25]:
despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62566 entries, 0 to 62565
Data columns (total 18 columns):
Ano                     62566 non-null object
CNPJ                    62566 non-null object
Deputado                62566 non-null object
Fornecedor              62566 non-null object
Matricula               62566 non-null int64
Mes                     62566 non-null int64
Tipo                    62566 non-null object
Valor                   62566 non-null float64
cnpj_raiz_fornecedor    62566 non-null object
codigo_unico            62566 non-null object
nome_deputado           62566 non-null object
uf                      62566 non-null object
nome_completo           62566 non-null object
sequencial              62566 non-null object
cpf                     62566 non-null object
nome_urna               62566 non-null object
partido_eleicao         62566 non-null object
situacao                62566 non-null object
dtypes: float64(1), int64(2), object(15)
memory usage: 8.6+ MB


In [26]:
despesas.reset_index().head()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
0,0,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,3,A - COMBUSTÍVEIS E LUBRIFICANTES,200.00,71806251,52,ABELARDO CAMARINHA,SP,JOSE ABELARDO GUIMARÃES CAMARINHA,250000001157,38233754820,ABELARDO CAMARINHA,PSB,ELEITO POR QP
1,1,2015,68064740000125,ABELARDO CAMARINHA,PAPER FACE COMERCIAL LTDA,300257,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,295.40,68064740,52,ABELARDO CAMARINHA,SP,JOSE ABELARDO GUIMARÃES CAMARINHA,250000001157,38233754820,ABELARDO CAMARINHA,PSB,ELEITO POR QP
2,2,2015,22145388877,ABELARDO CAMARINHA,LARA SERVINO VARGAS ABREU,300257,3,N - MORADIA,"2,850.00",documento_cpf_,52,ABELARDO CAMARINHA,SP,JOSE ABELARDO GUIMARÃES CAMARINHA,250000001157,38233754820,ABELARDO CAMARINHA,PSB,ELEITO POR QP
3,3,2015,10915883000113,ABELARDO CAMARINHA,CENTRO AUTOMOTIVO TOP 23 LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,100.00,10915883,52,ABELARDO CAMARINHA,SP,JOSE ABELARDO GUIMARÃES CAMARINHA,250000001157,38233754820,ABELARDO CAMARINHA,PSB,ELEITO POR QP
4,4,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,195.01,71806251,52,ABELARDO CAMARINHA,SP,JOSE ABELARDO GUIMARÃES CAMARINHA,250000001157,38233754820,ABELARDO CAMARINHA,PSB,ELEITO POR QP


In [27]:
despesas.rename(columns={'Valor':'valor_fornecedor'}, inplace=True)

In [28]:
def limpeza3(linha):
    documento = linha["CNPJ"]
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf_"

In [30]:
despesas['cnpj_raiz_fornecedor'] = despesas.apply(limpeza3, axis = 1)

Total de despesas

In [31]:
"${:,.2f}".format(despesas.valor_fornecedor.sum())

'$57,102,331.31'

In [32]:
despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62566 entries, 0 to 62565
Data columns (total 18 columns):
Ano                     62566 non-null object
CNPJ                    62566 non-null object
Deputado                62566 non-null object
Fornecedor              62566 non-null object
Matricula               62566 non-null int64
Mes                     62566 non-null int64
Tipo                    62566 non-null object
valor_fornecedor        62566 non-null float64
cnpj_raiz_fornecedor    62566 non-null object
codigo_unico            62566 non-null object
nome_deputado           62566 non-null object
uf                      62566 non-null object
nome_completo           62566 non-null object
sequencial              62566 non-null object
cpf                     62566 non-null object
nome_urna               62566 non-null object
partido_eleicao         62566 non-null object
situacao                62566 non-null object
dtypes: float64(1), int64(2), object(15)
memory usage: 8.6+ MB


Despesas só com CNPJ

In [33]:
seleciona_cnpjs = despesas[(despesas['cnpj_raiz_fornecedor'] != 'documento_cpf_')]
"${:,.2f}".format(seleciona_cnpjs.valor_fornecedor.sum())

'$46,163,628.21'

# Doações

Cruza para pegar todas doações diretas e não repetir fornecedores

In [34]:
despesas_doadores_diretos = pd.merge(eleitos_d_doadores, despesas.drop_duplicates('cnpj_raiz_fornecedor'), left_on='cnpj_doador_raiz', right_on='cnpj_raiz_fornecedor')

In [35]:
despesas_doadores_diretos.reset_index().head()

,index,uf_x,partido_eleicao_x,cargo,nome_completo_x,cpf_x,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,...,cnpj_raiz_fornecedor,codigo_unico_y,nome_deputado_y,uf,nome_completo,sequencial_y,cpf_y,nome_urna_y,partido_eleicao,situacao_y
0,0,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,46319901000141,JOIA DE GUARULHOS POSTO DE SERVIÇOS LTDA,"1,000.00",Recursos de pessoas jurídicas,...,46319901,4928,CARLOS NEDER,SP,CARLOS ALBERTO PLETZ NEDER,250000000518,01309189897,CARLOS NEDER,PT,ELEITO POR MÉDIA
1,1,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,45827425000107,SUPERMERCADOS IRMÃOS LOPES S/A,"10,000.00",Recursos de pessoas jurídicas,...,45827425,434,ANALICE FERNANDES,SP,ANALICE FERNANDES,250000001534,03669559850,ANALICE FERNANDES,PSDB,ELEITO POR QP
2,2,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,60437647000107,SILVA E BARBOSA COM ALIM LTDA,"5,000.00",Recursos de pessoas jurídicas,...,60437647,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
3,3,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,60437647000107,SILVA E BARBOSA COMERCIO DE ALIMENTOS LTDA,"5,000.00",Recursos de pessoas jurídicas,...,60437647,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
4,4,SP,PSDB,Deputado Estadual,JOAO CARLOS CARAMEZ,41352009820,67376434000161,AUTO POSTO NOVA ITAPEVI LTDA,"3,074.02",Recursos de pessoas jurídicas,...,67376434,268,BARROS MUNHOZ,SP,JOSE ANTONIO BARROS MUNHOZ,250000001557,03795640849,BARROS MUNHOZ,PSDB,ELEITO POR QP


Total de doações dos doadores que são fornecedores

In [36]:
"${:,.2f}".format(despesas_doadores_diretos.valor_doado.sum())

'$990,072.30'

Quem mais doou entre os fornecedores

In [38]:
despesas_doadores_diretos.groupby("nome_doador").valor_doado.sum().reset_index().sort_values("valor_doado", ascending=False).head(10)

,nome_doador,valor_doado
62,UNIQUE - SERVIÇOS DE HOTELARIA E ALIM.· COM. E...,"100,000.00"
10,AUTO POSTO POLISERVIÇOS I LTDA,"100,000.00"
60,TENDA ATACADO LTDA.,"100,000.00"
43,PORTO SEGURO COMPANHIA DE SEGUROS GERAIS,"75,000.00"
16,COOP - COOPERATIVA DE CONSUMO,"60,000.00"
40,ORGANIZACAO COMERCIAL LAGOA AZUL LTDA,"60,000.00"
14,CIA BRASILEIRA DE DISTRIBUIÇÃO,"50,000.00"
17,COOP. PLANTADORES DE CANA DO OESTE DO EST DE S...,"50,000.00"
18,COOPERATIVA DOS PLANTADORES DE CANA DO OESTE D...,"50,000.00"
6,AUTO POSTO EXPRESS DE DRACENA LTDA,"40,000.00"


Soma pelo cnpj_doador_raiz

In [39]:
despesas_doadores_diretos.groupby("cnpj_doador_raiz").valor_doado.sum().reset_index().sort_values("valor_doado", ascending=False).head(10)

,cnpj_doador_raiz,valor_doado
2,01157555,"100,000.00"
4,03109168,"100,000.00"
42,55001879,"100,000.00"
55,71320915,"100,000.00"
48,61198164,"75,000.00"
58,72911837,"60,000.00"
45,57508426,"60,000.00"
36,47508411,"50,000.00"
25,14178235,"40,000.00"
15,05800256,"40,000.00"


In [40]:
nome = despesas_doadores_diretos[(despesas_doadores_diretos['cnpj_doador_raiz'] == '01157555')]
nome

,uf_x,partido_eleicao_x,cargo,nome_completo_x,cpf_x,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,cnpj_raiz_fornecedor,codigo_unico_y,nome_deputado_y,uf,nome_completo,sequencial_y,cpf_y,nome_urna_y,partido_eleicao,situacao_y
135,SP,PSDB,Deputado Estadual,ORLANDO MORANDO JUNIOR,17849486838,01157555001186,TENDA ATACADO LTDA.,"100,000.00",Recursos de pessoas jurídicas,Nao especificado,...,01157555,442,AFONSO LOBATO,SP,JOSÉ AFONSO LOBATO,250000001043,47965282968,AFONSO LOBATO,PV,ELEITO POR QP


In [41]:
nome = despesas_doadores_diretos[(despesas_doadores_diretos['cnpj_doador_raiz'] == '03109168')]
nome

,uf_x,partido_eleicao_x,cargo,nome_completo_x,cpf_x,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,cnpj_raiz_fornecedor,codigo_unico_y,nome_deputado_y,uf,nome_completo,sequencial_y,cpf_y,nome_urna_y,partido_eleicao,situacao_y
138,SP,PP,Deputado Estadual,ANTONIO ASSUNÇÃO DE OLIM,02953727884,03109168000128,UNIQUE - SERVIÇOS DE HOTELARIA E ALIM.· COM. E...,"100,000.00",Recursos de pessoas jurídicas,Nao especificado,...,03109168,13093,MARCOS DAMASIO,SP,MARCOS ROBERTO DAMÁSIO DA SILVA,250000000739,05783796867,MARCOS DAMASIO,PR,ELEITO POR QP


Tabela dinâmica com ano, deputado e fornecedores - somando doacoes - (não precisaria de ano)

In [42]:
pd.pivot_table(despesas_doadores_diretos,index=["Ano","Deputado", "Fornecedor"], values=["valor_doado"], aggfunc=np.sum)

valor_doado
Ano  Deputado              Fornecedor                                                     
2015 ADILSON ROSSI         KALUNGA COM INDUSTRIA GRAFICA LTDA                    10,540.00
     AFONSO LOBATO         GRAFICA ELITE                                             70.00
                           TENDA ATACADO                                        100,000.00
     ANALICE FERNANDES     SUPERMERCADOS IRMAOS LOPES S/A                        10,000.00
     ANGELO PERUGINI       GOOD BOM SUPERMERCADOS LTDA                            2,864.25
     BARROS MUNHOZ         SANT'ANA ACABAM. GRÁFICOS E EMBAL. LTDA.                 600.00
     CAMPOS MACHADO        NOVA DISTRIBUIDORA DE VEICULOS LTDA                    2,500.00
     CARLOS NEDER          JOIA DE GUARULHOS DE SERVICOS LTDA                     1,000.00
                           POSTO ALVORADA DE FRANCA LTDA                         10,000.00
     CELINO CARDOSO        AUTO POSTO LAERCIO DE MAIRIPORÃ LTDA                   3,551.31
     CELSO GIGLIO          GRÁFICA TRES PONTAS LTDA - EPP                         1,210.00
     CELSO NASCIMENTO      POSTO JENNER LTDA                                      1,000.00
     CHICO SARDELLI        CRONY ARTES GRAFICAS LTDA ME                             460.00
                           JOSE H. G. DE OLIVEIRA EDITORA ME                      1,260.00
     DILADOR BORGES        SOLUÇÃO IMPRESSA GRAFICA E EDITORA LTDA ME               600.00
     ED THOMAS             HANNA INCORPORAÇÕES E VENDAS LTDA                     15,000.00
     GERALDO CRUZ          PASARGADA SERVIÇOS AUTOMOTIVOS LTDA                    3,000.00
     GILENO GOMES          PORTO SEGURO CIA DE SEGUROS GERAIS                    75,000.00
     GILSON DE SOUZA       ALGAR TELECOM S/A                                        160.00
     IGOR SOARES           ALPHAMARK PROPAGANDA E MARKETING LTDA                  2,000.00
                           AUTO POSTO POLISERVIÇOS I LTDA                       100,000.00
     ITAMAR BORGES         FERNANDO LUIZ LAUDINO & CIA LTDA - ME                  2,700.00
     LUIZ CARLOS GONDIM    AUTO POSTO B B MOGI LTDA                               2,324.25
                           GRANJA VIANA AUTO POSTO EIRELI                         1,000.00
                           INTEGRA EDITORA GRÁFICA LTDA EPP                         450.00
     ORLANDO MORANDO       EXPRESSÃO SANTO ANDRE GRAFICA E EDITORA LTDA EPP         325.00
     REINALDO ALGUZ        AUTO POSTO EXPRESS DE DRACENA LTDA                    40,000.00
     SARAH MUNHOZ          UAU DIGITAL COMUICAÇÃO LTDA                            2,750.00
     TEONILIO BARBA        REDE SEG GRAFICA E EDITORA EIRELI EPP                    100.00
2016 ADILSON ROSSI         CIA BRASILEIRA DE DISTRIBUIÇÃO                        50,000.00
     ANALICE FERNANDES     ORGANIZAÇAO COML. LAGO AZUL LTDA                      60,000.00
                           SUPERMERCADOS KACULA LTDA                             10,000.00
     ANGELO PERUGINI       ASPIL INFORMATICA LTDA EPP                            10,400.00
                           COMERCIO DE COMBUSTIVEIS PARQUE GABRIEL LTDA          25,000.00
     ATILA JACOMUSSI       COOP COOPERATIVA DE CONSUMO                           60,000.00
     BARROS MUNHOZ         AUTO POSTO NOVA ITAPEVI LTDA                           3,074.02
                           POSTO FRANCA CLARAVAL                                 10,000.00
                           SUPERMERCADOS CAVICCHIOLLI LTDA.                       2,994.27
     CARLOS NEDER          AUTO POSTO PAGUE MENOS LTDA                            8,493.20
     CORONEL TELHADA       CENTRO AUTOMOTIVO ELOS DE OURO LTDA                    1,000.00
     DAVI ZAIA             COOP. PLANT. DE CANA OESTE EST. SÃO PAULO            100,000.00
     FERNANDO CURY         HABIBS BOTUCATU                                        1,500.00
                           PROVIDE COMERCIO DE VEICULOS BOTUCATU LTDA      

In [43]:
despesas_doadores_diretos.to_csv('resultados/doacoes_deputados_sp_fornecedores.csv', index=False, decimal = ',')

# PF

In [45]:
despesas_PF = despesas[(despesas['cnpj_raiz_fornecedor'] == 'documento_cpf_')]

In [47]:
despesas_PF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4942 entries, 2 to 62565
Data columns (total 18 columns):
Ano                     4942 non-null object
CNPJ                    4942 non-null object
Deputado                4942 non-null object
Fornecedor              4942 non-null object
Matricula               4942 non-null int64
Mes                     4942 non-null int64
Tipo                    4942 non-null object
valor_fornecedor        4942 non-null float64
cnpj_raiz_fornecedor    4942 non-null object
codigo_unico            4942 non-null object
nome_deputado           4942 non-null object
uf                      4942 non-null object
nome_completo           4942 non-null object
sequencial              4942 non-null object
cpf                     4942 non-null object
nome_urna               4942 non-null object
partido_eleicao         4942 non-null object
situacao                4942 non-null object
dtypes: float64(1), int64(2), object(15)
memory usage: 733.6+ KB


In [48]:
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    if len(linha['CNPJ']) < 11:
        conta = 11 - (len(linha['CNPJ']))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + linha['CNPJ']
    elif len(linha['CNPJ']) == 11:
        alterado = linha['CNPJ']
    
    return alterado

In [49]:
nova = despesas_PF.apply(compara1, axis=1) #lê dataframe e chama a função

despesas_PF['cpf_inteiro'] = nova

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
despesas_doadores_diretosPF = pd.merge(eleitos_d_doadores, despesas_PF.drop_duplicates('cpf_inteiro'), left_on='cpf_cnpj_doador', right_on='cpf_inteiro')

Total de doações de fornecedores PF

In [51]:
"${:,.2f}".format(despesas_doadores_diretosPF.valor_doado.sum())

'$421,744.00'

Maiores doadores PF que são fornecedores

In [52]:
despesas_doadores_diretosPF.groupby("nome_doador").valor_doado.sum().reset_index().sort_values("valor_doado", ascending=False)

,nome_doador,valor_doado
6,ROBERTO MASSAFERA,"315,520.00"
4,IGOR SOARES EBERT,"63,000.00"
3,ELEICAO 2014 ULYSSES MARIO TASSINARI DEPUTADO ...,"16,500.00"
0,ANTONIO CABEZA SASTRE,"10,500.00"
2,CLOVIS SILES GALVÃO,"10,500.00"
5,MARIA ELIZA INTERDONATO NUNES VIANA,"3,000.00"
7,ROBERTO TAVARES DE CARVALHO,"2,000.00"
1,CLEUSA ANANIAS DE ARAUJO,724.00


ROBERTO MASSAFERA é um deputado, fornecedor e doou para si mesmo

In [53]:
nome = despesas_doadores_diretosPF[(despesas_doadores_diretosPF['nome_doador'] == 'ROBERTO MASSAFERA')]
nome

,uf_x,partido_eleicao_x,cargo,nome_completo_x,cpf_x,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,codigo_unico_y,nome_deputado_y,uf,nome_completo,sequencial_y,cpf_y,nome_urna_y,partido_eleicao,situacao_y,cpf_inteiro
5,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"100,000.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
6,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"20,000.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
7,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"30,000.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
8,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"2,000.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
9,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,640.00,Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
10,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"2,880.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
11,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"10,000.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
12,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"30,000.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800
13,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,"120,000.00",Recursos próprios,Nao especificado,...,10622,ROBERTO MASSAFERA,SP,ROBERTO MASSAFERA,250000001548,02674980800,ROBERTO MASSAFERA,PSDB,ELEITO POR QP,02674980800


# Despesas

Pega todos os valores de fornecedores que são doadores

In [55]:
despesas_doadores_diretos_f = pd.merge(eleitos_d_doadores.drop_duplicates('cnpj_doador_raiz'), despesas, left_on='cnpj_doador_raiz', right_on='cnpj_raiz_fornecedor')

Soma de despesas pagas aos fornecedores/doadores 

In [56]:
"${:,.2f}".format(despesas_doadores_diretos_f.valor_fornecedor.sum())

'$1,856,363.82'

Tabela dinâmica por ano, deputado, cnpj raiz e fornecedor

In [57]:
pd.pivot_table(despesas_doadores_diretos_f,index=["Ano","Deputado", "cnpj_raiz_fornecedor", "Fornecedor"], values=["valor_fornecedor"], aggfunc=np.sum)

valor_fornecedor
Ano  Deputado              cnpj_raiz_fornecedor Fornecedor                                                          
2015 ADILSON ROSSI         43283811             KALUNGA COM INDUSTRIA GRAFICA LTDA                            922.59
     AFONSO LOBATO         01157555             TENDA ATACADO                                                 170.50
                           47508411             PÃO DE AÇÚCAR                                                  10.44
                           56035009             GRAFICA ELITE                                              29,200.00
     ALDO DEMARCHI         43283811             KALUNGA COMÉRCIO E INDUSTRIA GRÁFICA LTDA                     493.70
                                                KALUNGA COMÉRCIOINDUSTRIA GRÁFICA LTDA                      1,334.78
     ALENCAR SANTANA BRAGA 43283811             KALUNGA LTDA.                                                 264.90
     ALEXANDRE PEREIRA     43283811             KALUNGA  COM E IND. GRAF. LTDA -SP                            585.63
                                                KALUNGA COM  IND. GRAFICA LTDA                                327.90
                                                KALUNGA COM IND GRAFICA LTDA                                   95.70
                                                KALUNGA COMERCIO E IND GRAFICA LTDA                            39.00
                                                KALUNGA TATUAPÉ                                                37.80
                                                KALUNGA VILA GUILHERME                                        199.70
     ANA DO CARMO          43283811             KALUNGA                                                       805.99
     ANALICE FERNANDES     43283811             KALUNGA COMERCIO E IND. GRAFICA LTDA                          159.90
                           45827425             SUPERMERCADOS IRMAOS LOPES S/A                                139.49
                           47508411             CIA BRASILEIRA DE DISTRIBUIÇÃO                                266.18
     ANGELO PERUGINI       43283811             KALUNGA COMÉRCIO E IND. GRAFICA LTDA                          830.16
                           56794852             GOOD BOM SUPERMERCADOS LTDA                                    44.93
     ANTONIO SALIM CURIATI 43283811             KALUNGA COMERCIO E IND. GRAFICA LTDA                        2,131.32
                                                KALUNGA COMERCIO E IND. GRAFICA LTDA.                         539.97
     ATILA JACOMUSSI       43283811             KALUNGA COMERCIO E IND. GRÁFICA LTDA                        4,324.52
                           47508411             CIA BRASILEIRA DE DISTRIBUIÇÃO                                210.84
     BARROS MUNHOZ         03737451             SANT'ANA ACABAM. GRÁFICOS E EMBAL. LTDA.                    1,400.00
                           43283811             KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA                     863.68
                                                KALUNGA COMÉRCIO E IND.GRÁFICA LTDA.                          652.60
                                                KALUNGA COMÉRCIO INDÚSTRIA GRÁFICA LTDA.                      631.30
                           47508411             CIA BRASILEIRA D EDISTRIBUIÇÃO                                 18.06
                                                CIA BRASILEIRA DE DISTRIBUIÇÃO                                236.27
                           72911837             ORGANIZAÇÃO COML. LAGO AZUL LTDA.                             165.03
...                                                                                                              ...
2018 RICARDO MADALENA      43283811             KALUNGA                                                     1,072.25
                                                KALUNGA COM. E IND. GRÁFICA LTDA                              471.49
     

Soma de despesas pelo CNPJ raiz dos fornecedores

In [59]:
despesas_doadores_diretos_f.groupby("cnpj_raiz_fornecedor").valor_fornecedor.sum().reset_index().sort_values("valor_fornecedor", ascending=False).head(10)

,cnpj_raiz_fornecedor,valor_fornecedor
30,43283811,"392,393.65"
27,15045533,"232,065.00"
40,54450705,"227,912.40"
37,47828827,"215,196.00"
9,04867562,"127,048.40"
38,52837408,"115,000.00"
57,72855505,"104,266.79"
14,05790610,"94,980.58"
43,56035009,"67,080.00"
0,00158540,"57,750.00"


In [61]:
nome = despesas_doadores_diretos_f[(despesas_doadores_diretos_f['cnpj_raiz_fornecedor'] == '43283811')]
nome.head(10)

,uf_x,partido_eleicao_x,cargo,nome_completo_x,cpf_x,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,cnpj_raiz_fornecedor,codigo_unico_y,nome_deputado_y,uf,nome_completo,sequencial_y,cpf_y,nome_urna_y,partido_eleicao,situacao_y
282,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
283,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
284,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
285,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
286,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
287,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
288,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
289,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
290,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
291,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,"2,635.00",Recursos de pessoas jurídicas,Nao especificado,...,43283811,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA


Deputados com a Kalunga

In [62]:
nome['Deputado'].value_counts()

GIL LANCASTER                        69
DELEGADO OLIM                        47
PEDRO TOBIAS                         42
ANTONIO SALIM CURIATI                41
JORGE CARUSO                         36
BARROS MUNHOZ                        35
RICARDO MADALENA                     33
LUIZ CARLOS GONDIM                   33
GILENO GOMES                         33
MARTA COSTA                          31
RAUL MARCELO                         29
WELLINGTON MOURA                     28
TEONILIO BARBA                       27
LUIZ TURCO                           27
CELINO CARDOSO                       26
JORGE WILSON XERIFE DO CONSUMIDOR    26
ANALICE FERNANDES                    26
CARLOS BEZERRA JR.                   24
VAZ DE LIMA                          23
FERNANDO CAPEZ                       22
CARLOS NEDER                         21
PAULO CORREA JR                      21
RITA PASSOS                          21
ORLANDO MORANDO                      21
EDSON GIRIBONI                       20


Soma de despesas pelo nome do fornecedor/doador

In [63]:
despesas_doadores_diretos_f.groupby("Fornecedor").valor_fornecedor.sum().reset_index().sort_values("valor_fornecedor", ascending=False).head(10)

,Fornecedor,valor_fornecedor
2,ALPHAMARK SOLUÇÕES LTDA,"148,545.00"
58,EXPRESSÃO SANTO ANDRE GRAFICA E EDITORA LTDA EPP,"144,000.00"
68,GRÁFICA TRES PONTAS LTDA - EPP,"134,810.00"
55,CRONY ARTES GRAFICAS LTDA ME,"127,048.40"
70,HANNA INCORPORAÇÕES E VENDAS LTDA,"115,000.00"
60,FERNANDO LUIZ LAUDINO & CIA LTDA - ME,"94,980.58"
66,GRAFICA TRES PONTAS LTDA - EPP,"93,102.40"
153,NOVA DISTRIBUIDORA DE VEICULOS LTDA,"90,316.01"
1,ALPHAMARK PROPAGANDA E MARKETING LTDA,"83,520.00"
57,EXPRESSAO SANTO ANDRE GRAFICA EDITORA LTDA EPP,"71,196.00"


In [64]:
despesas_doadores_diretos_f.to_csv('resultados/despesas_deputados_sp_fornecedores_doadores.csv', index=False, decimal = ',')

# PF

In [65]:
despesas_doadores_diretos_f_PF = pd.merge(eleitos_d_doadores.drop_duplicates('cpf_cnpj_doador'), despesas_PF, left_on='cpf_cnpj_doador', right_on='cpf_inteiro')

Soma de despesas dos doadores que são PF

In [66]:
"${:,.2f}".format(despesas_doadores_diretos_f_PF.valor_fornecedor.sum())

'$482,205.25'

In [67]:
despesas_doadores_diretos_f_PF.groupby("Fornecedor").valor_fornecedor.sum().reset_index().sort_values("valor_fornecedor", ascending=False)

,Fornecedor,valor_fornecedor
6,ROBERTO TAVARES DE CARVALHO,"180,410.94"
2,CLEUSA ANANIAS DE ARAUJO,"94,400.00"
0,ANTONIO CABEZA SASTRE,"65,540.00"
3,CLOVIS SILLES GALVÃO,"65,540.00"
4,MARIA ELISA INTERNODATO NUNES VIANA,"52,664.50"
5,MARIA ELISA INTERNODATO NUNES VIEIRA,"17,492.90"
7,ULYSSES MARIO TASSINARI,"3,656.91"
1,CIRO BERBES JUNIOR,"2,500.00"


In [68]:
despesas_doadores_diretos_f_PF.groupby("cpf_inteiro").valor_fornecedor.sum().reset_index().sort_values("valor_fornecedor", ascending=False)

,cpf_inteiro,valor_fornecedor
6,16381726849,"180,410.94"
0,01530098882,"94,400.00"
3,02698706872,"65,540.00"
4,11187492868,"65,540.00"
5,15737946860,"52,664.50"
7,28223302802,"17,492.90"
1,01788949820,"3,656.91"
2,02674980800,"2,500.00"


In [69]:
nome = despesas_doadores_diretos_f_PF[(despesas_doadores_diretos_f_PF['Fornecedor'] == 'ROBERTO TAVARES DE CARVALHO')]
nome

,uf_x,partido_eleicao_x,cargo,nome_completo_x,cpf_x,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,codigo_unico_y,nome_deputado_y,uf,nome_completo,sequencial_y,cpf_y,nome_urna_y,partido_eleicao,situacao_y,cpf_inteiro
103,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
104,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
105,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
106,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
107,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
108,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
109,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
110,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
111,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849
112,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,"2,000.00",Recursos de pessoas físicas,Nao especificado,...,13113,GILENO GOMES,SP,GIRLÊNIO GOMES DE OLIVEIRA,250000000205,08620367803,GILENO GOMES,PSL,ELEITO POR QP,16381726849


In [70]:
despesas_doadores_diretos_f_PF.to_csv('resultados/despesas_deputados_sp_fornecedoresPF_doadores.csv', index=False, decimal = ',')